MIT License

Copyright (c) 2026 Eric Ahlqvist and Richard Pink

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

Authors: Eric Ahlqvist, Richard Pink

This gap worksheet concerns weak Schur \sigma-groups G of type (3,3) for p=3.
Its goal is to determine for which such G the subgroups D2(G), D3(G), resp.
D4(G) are powerful. All groups under consideration are finite 3-groups.

===============================================================================

Part 0: Setup

In [2]:
LoadPackage("AutPGrp");;
LoadPackage("anupq");;

===============================================================================

Part 1: Some utility functions

In [5]:
# Runtimes:
oldruntime := Runtime();;
# The following macro gives the runtime since the last invocation:
TimeUsed := function()
local timeused;
timeused := Runtime()-oldruntime;
oldruntime := Runtime();
return timeused;
end;;

# This computes the log size log_3|H| for a finite 3-group H:
LSize := function(H)
return LogInt(Size(H),3);
end;;


The Zassenhaus filtration:

For any pro-3-group $G$ we denote by $K_i(G)$ its descending central series, defined by $$K_1 := G \quad\text{ and }\quad  K_{i+1} := [G,K_i]$$  for all $i\ge 1$.
It is computed as a list by the GAP function LowerCentralSeriesOfGroup.

We denote by $\mathrm{℧}^{i}(P)$ the subgroup generated by all $p^i$-th powers 
of elements of $G$.  It is computed by the GAP function Agemo(..,p,i).

We denote by $D_i(G)$ the Zassenhaus filtration of $G$. For this we have $D_1=G$,
and $D_2$ is the Frattini subgroup, and the higher $D_i$ can be characterized by
$D_i(G) = \prod_{jp^k\ge i} \mathrm{℧}^{k}(K_j(G))$.

For any 3-group $G$ the following macro computes $D_2(G)$:

In [6]:
ComputeD2 := function(G)
return FrattiniSubgroup(G); 
end;;

For any 3-group $G$ the following macro computes $D_3(G)$:

In [7]:
ComputeD3 := function(G)
local gens,KG;
gens := GeneratorsOfGroup(Agemo(G,3,1));
KG := LowerCentralSeriesOfGroup(G);
if Length(KG) > 2
then gens := Concatenation(gens,GeneratorsOfGroup(KG[3]));
fi;
return Subgroup(G,gens);
end;;

Actually, in the case $p=3$ we already have $D_3(G)=\mathrm{℧}^1(G)$, but never mind.

For any 3-group $G$ the following macro computes $D4(G)$:

In [8]:
ComputeD4 := function(G)
local gens,KG;
gens := GeneratorsOfGroup(Agemo(G,3,2));
KG := LowerCentralSeriesOfGroup(G);
if Length(KG) > 2
then gens := Concatenation(gens,GeneratorsOfGroup(Agemo(KG[2],3)));
fi;
if Length(KG) > 4
then gens := Concatenation(gens,GeneratorsOfGroup(KG[4]));
fi;
return Subgroup(G,gens);
end;;

The lower central series:

As in ANUPQ (see its manual 2.1-2), the descending p-central series $P_i(G)$ of 
a $p$-group $G$ is defined by  $P_0 := G$  and  $P_{i+1} := P_i^p[G,P_i]$  for all $i\ge 0$.
It is computed as a list by the GAP function PCentralSeries(..,p).

Caution: Other sources, like core gap code (see Reference Manual 39.17-13) 
define the p-central series by  $U_1 := G$  and  $U_{i+1} := U_i^p[G,U_i]$, 
which effects a shift by 1. We stick to the convention in the ANUPQ package, 
also used in [Boston-Bush-Hajir 2017].

Following the ANUPQ manual 2.1-2, the $p$-class of $G$ is the least i with $P_i=1$. 
Thus the trivial group has $p$-class 0, and a non-trivial elementary abelian 
$p$-group has $p$-class 1.

This function computes the $p$-class of a 3-group $G$:

In [9]:
ComputePClass := function(G)
return Length(PCentralSeries(G,3))-1;
end;;

Can one use instead the function PqPClass from Section 5.5-4 
"interactive ANUPQ functions" of the ANUPQ manual?
And would that be faster?

For any function E which to any $p$-group $G$ associates a normal subgroup $E(G)$
we define subgroups $E_1(G) := D_2(E(G))$ and $E_2(G) := \mathrm{℧}^1(E(G))·[G,D_2(E(G))]$.

These satisfy $E_2(G) \subset E_1(G) \subset E(G)$, and we have determined that
$E(G)$ is powerful if and only if $E_2(G)=E_1(G)$ or equivalently $E_1(G/E_2(G))=1$.
The aim of this worksheet is to test this for different choices of $E$.

The following functions compute $E_2(G)$ and $E_1(G)$ for any given function $E$:

In [11]:
ComputeE1 := function(E,G)
return FrattiniSubgroup(E(G));
end;;

ComputeE2 := function(E,G)
local EG;
EG := E(G);
return Subgroup(G, Union(GeneratorsOfGroup(Agemo(EG,3)),
      GeneratorsOfGroup(CommutatorSubgroup(G,FrattiniSubgroup(EG)))));
end;;

===============================================================================

Part 2: Some computations for the free pro-3-group

Let $F_2$ denote the free pro-3-group on 2 generators.

First we compute its maximal quotients of p-class 3 resp. 4:

In [14]:
F2 := FreeGroup(2);;
F2ByP3 := Pq( F2: Prime:=3, ClassBound := 3);;
F2ByP4 := Pq( F2: Prime:=3, ClassBound := 4);;

As their quotients by their respective $D_4(..)$ have the same size, 

In [15]:
[LSize(F2ByP3/ComputeD4(F2ByP3)), 
 LSize(F2ByP4/ComputeD4(F2ByP4))];

[ 7, 7 ]

we deduce that $P_3(F_2) \subset D_4(F_2)$.

Next, comparing the orders of $D_3(F_2/P_3(F_2))$ and $P_2(F_2/P_3(F_2))\cdot D_3(F_2/P_3(F_2))$ shows that $P_2(F_2) \subset D_3(F_2)$:

In [17]:
P2F2ByP3 := PCentralSeries(F2ByP3,3)[3];;
[LSize(ComputeD3(F2ByP3)), 
 LSize(Subgroup(F2ByP3,Concatenation(GeneratorsOfGroup(P2F2ByP3),
                              GeneratorsOfGroup(ComputeD3(F2ByP3)))))];

[ 7, 7 ]

Likewise comparing the orders of $P_2(F_2/P_3(F_2))$ and $P_2(F_2/P_3(F_2))\cdot D_4(F_2/P_3(F_2))$ shows that $D_4(F_2) \subset P_2(F_2)$:

In [18]:
[LSize(P2F2ByP3), 
 LSize(Subgroup(F2ByP3,Concatenation(GeneratorsOfGroup(P2F2ByP3),
                              GeneratorsOfGroup(ComputeD4(F2ByP3)))))];

[ 5, 5 ]

For every factor group $G$ of $F_2$ it now also follows that 
$D_2(G) \supset D_3(G) \supset P_2(G) \supset D_4(G) \supset P_3(G)$.

The following group orders imply that these are proper inclusions for $G=F_2$:

In [19]:
[LSize(ComputeD2(F2ByP3)),
 LSize(ComputeD3(F2ByP3)),
 LSize(P2F2ByP3),
 LSize(ComputeD4(F2ByP3))];

[ 8, 7, 5, 3 ]

Likewise the following shows that for $E:=D_2$ we have $E_1(F_2) \subset D_4(F_2)$:

In [21]:
E1F2ByP3 := ComputeE1(ComputeD2,F2ByP3);;
[LSize(ComputeD4(F2ByP3)), 
 LSize(Subgroup(F2ByP3,Concatenation(GeneratorsOfGroup(E1F2ByP3),
                              GeneratorsOfGroup(ComputeD4(F2ByP3)))))];

[ 3, 3 ]

For any $E\subset D_2$ this implies that $E_2(G) \subset E_1(G) \subset D_4(G)$.

===============================================================================

Part 3: Recognizing $\sigma$-3-groups with precisely 2 odd generators

The ANUPQ-package only deals with $p$-groups, and one would have 
to adapt its source code in order to deal with $\sigma$-p-groups.

But it is known that for any pro-$p$-group $G$, any two automorphisms 
of order 2 that act by $-1$ on $G/D_2(G)$ are conjugate under $\operatorname{Aut(G)}$.
Thus the isomorphism class of a \sigma-pro-p-group with odd generators
is determined by the isomorphism class of the underlying pro-p-group.

Thus it will suffice to test for the existence of such an automorphism. 
The following macro does this for any p-group and any odd prime $p$.

In [22]:
IsOurSigmaGroup := function(G)
local A,Agens,pi,Gbar,Gbarbasis,d,lift,i,Abargens,phibarimages,j,
      AutGbar,Abar,sigma,Asigbar;
A := AutomorphismGroup(G);
# If A has odd order, there is no such automorphism:
if (Size(A) mod 2) <> 0
then return false;
else
# Else set Gbar := G/Fratt(G):
pi := NaturalHomomorphismByNormalSubgroupNC(G,FrattiniSubgroup(G));
Gbar := ImagesSource(pi);
Gbarbasis := MinimalGeneratingSet(Gbar);
d := Length(Gbarbasis);
# Construct a section Gbar --> G:
lift := [1..d];
for i in [1..d] do
lift[i] := PreImagesRepresentative(pi,Gbarbasis[i]);
od;
# For any generator phi of A construct the induced automorphism phibar of Gbar:
Agens := GeneratorsOfGroup(A);
Abargens := [1..Length(Agens)];
for i in [1..Length(Agens)] do
phibarimages := [1..d];
for j in [1..d] do
phibarimages[j] := ImageElm(pi,ImageElm(Agens[i],lift[j]));
od;
Abargens[i] := GroupHomomorphismByImages(Gbar,Gbar,Gbarbasis,phibarimages);
od;
# Construct the image Abar of A in Aut(Gbar):
AutGbar := AutomorphismGroup(Gbar);
Abar := Subgroup(AutGbar,Abargens);
# Construct the subgroup generated by Abar and -1:
sigma := GroupHomomorphismByFunction(Gbar,Gbar,
   function(x) return x^-1; end);
Asigbar := Subgroup(AutGbar,Concatenation(Abargens,[sigma]));
# Compare their sizes to decide whether sigma lies in Abar:
if Size(Abar) = Size(Asigbar)
then return true;
else return false;
fi; fi;
end;;

===============================================================================

Part 4: Finding all isomorphism classes of $G/D_4(G)$.

For any weak Schur $\sigma$-group $G$ of Zassenhaus type $(3,3)$ the quotient 
$G/D_4(G)$ has order $3^5$. Using the inbuilt library of small groups, 
we find all isomorphism classes of such groups of order $3^5$,
and collect their indices in the list of all groups of order $3^5$.

In [27]:
All243 := AllSmallGroups(3^5);;
OurIndices := [];;
for i in [1..Length(All243)] do
H := All243[i];                                        # if |H|=3^5
if      Size(ComputeD2(H)) = 3^3                       # if |D2(H)|=3^3
then if Size(ComputeD3(H)) = 3^2                       # if |D3(H)|=3^2
then if Size(ComputeD4(H)) = 1                         # if |D4(H)|=1
then OurIndices := Concatenation(OurIndices,[i]);
fi; fi; fi; od;
Print("There are ",Length(OurIndices)," such groups up to isomorphism.");
Print("Their indices are ",OurIndices);

There are 13 such groups up to isomorphism.Their indices are 
[ 2, 3, 4, 5, 6, 7, 8, 9, 13, 14, 15, 17, 18 ]

So there are 13 such groups up to isomorphism.
Their indices are [2, 3, 4, 5, 6, 7, 8, 9, 13, 14, 15, 17, 18].

We create a list SchurByD4 such that SchurByD4[$i$] is the group with index $i$.

In [30]:
imax := OurIndices[Length(OurIndices)];
SchurByD4 := [1..imax];;
for i in OurIndices do SchurByD4[i] := All243[i]; od;

18

Here and below, we put any data depending on $i$ in OurIndices into 
position $X[i]$ of some list $X$. This requires that X has some irrelevant 
entries for $i$ not in OurIndices, but those will never be accessed.

Throughout the comments below, we abbreviate Hi := SchurByD4[i],
and we call a $p$-group $G$ of type [$d,i$] if $d$=|Hi| and $G/D_4(G) \cong H_i$.

Note that each Hi has p-class 2 or 3:

In [31]:
for i in OurIndices do
Print(ComputePClass(SchurByD4[i])," ");
od;

2 3 3 3 3 3 3 3 3 3 3 3 3 

Though not used in this worksheet, we also identify the quotients $G/D_4(G)$
for all weak Schur $\sigma$-groups $G$ with $d(G)=2$ which are not of type $(3,3)$:

In [36]:
All729 := AllSmallGroups(3^6);;
OurIndices1 := [];;
for i in [1..Length(All729)] do
H := All729[i];                                        # if |H|=3^6
if      Size(ComputeD2(H)) = 3^4                       # if |D2(H)|=3^4
then if Size(ComputeD3(H)) = 3^3                       # if |D3(H)|=3^3
then if Size(ComputeD4(H)) = 1                         # if |D4(H)|=1
then OurIndices1 := Concatenation(OurIndices1,[i]);
fi; fi; fi; od;
Print("There are ",Length(OurIndices1)," such groups up to isomorphism.");
Print("Their indices are ",OurIndices1);

There are 5 such groups up to isomorphism.Their indices are 
[ 9, 10, 11, 12, 26 ]

So there are 5 such groups up to isomorphism.
Their indices are [9, 10, 11, 12, 26].

In [41]:
All2187 := AllSmallGroups(3^7);;
OurIndices2 := [];;
for i in [1..Length(All2187)] do
H := All2187[i];                                       # if |H|=3^7
if      Size(ComputeD2(H)) = 3^5                       # if |D2(H)|=3^5
then if Size(ComputeD3(H)) = 3^4                       # if |D3(H)|=3^4
then if Size(ComputeD4(H)) = 1                         # if |D4(H)|=1
then OurIndices2 := Concatenation(OurIndices2,[i]);
fi; fi; fi; od;
Print("There are ",Length(OurIndices2)," such groups up to isomorphism.");
Print("Their indices are ",OurIndices2);

There are 1 such groups up to isomorphism.Their indices are [ 33 ]

So there is 1 such group up to isomorphism with index 33.

===============================================================================

Part 5: Finding all isomorphism classes of $G/P_2(G)$.

Since $D_4(F_2)$ is contained in $P_2(F_2)$, for any weak Schur $\sigma$-group $G$
of type $[243,i]$ we have $G/P_2(G) \cong H_i/P_2(H_i)$. We first compute all 
these isomorphism classes.

In [43]:
SchurByP2 := [1..imax];;
for i in OurIndices do
SchurByP2[i] := Pq(SchurByD4[i]: Prime:=3, ClassBound:=2);
od;;

Since each $H_i$ has $p$-class 2 or 3, each $G/P_2(G)$ has p-class 2.

Next we determine which of them are isomorphic. Their orders are:

In [44]:
for i in OurIndices do Print(LSize(SchurByP2[i])," "); od;

5 3 3 3 3 3 3 3 4 4 4 4 4 

Thus the groups $G/P_2(G)$ fall into at least three isomorphism classes.
Checking which are isomorphic, we find that there are precisely three:

In [69]:
OurIndicesSubtypes := [[2],[3,4,5,6,7,8,9],[13,14,15,17,18]];;
for i in OurIndicesSubtypes[2] do
Print(IsPqIsomorphicPGroup(SchurByP2[i],SchurByP2[3])," ");
od;;
Print("\n\nFor the indices ",OurIndicesSubtypes[2],
      " the groups Hi/P2(Hi) are all isomorphic.\n");;
for i in OurIndicesSubtypes[3] do
Print(IsPqIsomorphicPGroup(SchurByP2[i],SchurByP2[13])," ");
od;;
Print("\n\nFor the indices ",OurIndicesSubtypes[3],
      " the groups Hi/P2(Hi) are all isomorphic.");;

true true true true true true true 

For the indices [ 3, 4, 5, 6, 7, 8, 9 
 ] the groups Hi/P2(Hi) are all isomorphic.
true true true true true 

For the indices [ 13, 14, 15, 17, 18 
 ] the groups Hi/P2(Hi) are all isomorphic.

This gives us the reduced list of the groups $H_i/P_2(H_i)$:

In [70]:
SchurByP2Red := [SchurByP2[2],SchurByP2[3],SchurByP2[13]];;

===============================================================================

Part 6: Finding all isomorphism classes of $G/P_3(G)$

For each $H_i$ we compute a list SchurByP[i][3] of all groups $G/P_3(G)$ 
up to isomorphism for all weak Schur $\sigma$-groups G of type $[243,i]$. 

First recall that for any $p$-group $H$ of $p$-class $j>0$, any $p$-group $K$ with
$K/P_j(K) \cong H$ is called a descendant of $H$. If in addition $K$ has $p$-class 
$j+1$, then $K$ is called an immediate descendant of $H$. The only function
that efficiently computes isomorphism classes of central extensions
seems to be the descendant function from the ANUPQ package. 

Now observe that for any Schur $\sigma$-group $G$ of type $(3,3)$ we have 
inclusions $P_3(G) \subset D_4(G) \subset P_2(G)$, which may be proper. 
Thus we have surjections $G/P_3(G) \twoheadrightarrow G/D_4(G) \cong H_i \twoheadrightarrow G/P_2(G)$,
which in general are not isomorphisms. In particular $G/P_3(G)$ is
a central extension of Hi, but in general not a descendant of $H_i$.

But $G/P_3(G)$ is a descendant of $G/P_2(G) \cong H_i/P_2(H_i)$.
We therefore first compute all descendants of $H_i/P_2(H_i)$ 
of $p$-class $\le 3$ and then identify the correct ones.

For each group H in the reduced list SchurByP2Red we now find all $G/P_3(G)$
up to isomorphism for all weak Schur $\sigma$-groups $G$ with $G/P_2(G) \cong H$:

In [72]:
Candidates := [1..3];;
for k in [1..3] do 
Candidates[k] := [];
# Take all descendants of Hi/P2(Hi) of p-class \le3:
KList := PqDescendants(SchurByP2Red[k]: ClassBound:=3);
# Since the ANUPQ function PqDescendants only computes proper descendants,
# we must include Hi/P2(Hi) separately:
KList := Concatenation(KList,[SchurByP2Red[k]]);
for K in KList do
# Keep only those for which K/D4(K) has the correct size:
if Size(K/ComputeD4(K)) = 3^5 then 
# Keep only those whose minimal number of relations as a factor group
# of F2/P3(F2) is 2. [See Section 6 of the article for the justification.]
if ComputePClass(K) = 3
then relEK := MultiplicatorRank(K) - NuclearRank(K);
else relEK := MultiplicatorRank(K);
fi;
if relEK = 2 then
# Keep only those which are \sigma-groups generated by odd elements:
if IsOurSigmaGroup(K) then
# Since the group Hi is already defined as a factor group of F2/D4(F2)
# by 2 odd relations, it follows that the 2 relations defining K
# as a factor group of F2/P3(F2) are also odd. Thus K is indeed 
# isomorphic to G/P3(G) for a weak Schur \sigma-group G.
Candidates[k] := Concatenation(Candidates[k],[K]);
fi; fi; fi; 
od; od;

Now we assign the candidates $K$ to their type $[243,i]$:

In [75]:
SchurByP := [1..imax];;
for i in OurIndices do 
SchurByP[i] := [1..99];
SchurByP[i][3] := [];
od;
for k in [1..3] do
for K in Candidates[k] do
for i in OurIndicesSubtypes[k] do
if IsIsomorphicPGroup(K/ComputeD4(K),SchurByD4[i])
then SchurByP[i][3] := Concatenation(SchurByP[i][3],[K]);
break;
fi; od; od; od;

See the lengths:

In [77]:
Print("Number of groups at level 3:  ");
for i in OurIndices do Print(Length(SchurByP[i][3])," "); od;

Number of groups at level 3:  10 1 1 1 1 1 1 1 3 3 3 2 3 

Check that all groups in SchurByP[i][3] have p-class 3:

In [79]:
for i in OurIndices do
for K in SchurByP[i][3] do
Print(ComputePClass(K)," ");
od; od;
Print("The initialization has so far used ",TimeUsed()," milliseconds.");

3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 
3 The initialization has so far used 10979 milliseconds.

===============================================================================

Part 7: Finding isomorphism classes of $G/P_j(G)$ for $j \ge 3$.

For each of our types $[243,i]$ and any $j\ge 3$ within a suitable range, 
we compute a list SchurByP[i][j] of all groups $K$ up to isomorphism with

--- $K \cong G/P_j(G)$ for some weak Schur $\sigma$-groups $G$ of type $[243,i]$, and

--- $K$ has $p$-class equal to $j$.

For $j=3$ we already have such a list SchurByP[i][3]. Given that list 
for some $j$, we therefore want a macro computing that list for $j+1$.

For later use we also keep track of the descendant relation between these 
groups. In the induction step we therefore create two more lists such that
for any indices $m$ and $n$ the following three statements are equivalent:

--- The group SchurByP[i][j+1][n] is a descendant of SchurByP[i][j][m].

--- We have SchurByPParent[i][j+1][n] = m.

--- The index n is contained in the list SchurByPChildren[i][j][m].

With this information we can always go up and down the descendant tree.
In particular, the group SchurByP[i][j][m] is a terminal node in that
tree if and only if the list SchurByPChildren[i][j][m] is empty.
Using Section 6 of the article one can show that the terminal nodes 
at level j are precisely the finite Schur $\sigma$-groups of $p$-class $j$ 
and of the given type $[243,i]$.

Initialization:

In [82]:
SchurByPParent := [1..imax];;
SchurByPChildren := [1..imax];;
for i in OurIndices do 
SchurByPParent[i] := [1..99];
SchurByPChildren[i] := [1..99];
for j in [1..99] do
SchurByPParent[i][j] := [];
SchurByPChildren[i][j] := [];
od; od;

The following macros assume that SchurByP[i] is a list of length $j$,
whose entry SchurByP[i][j] is already a list of groups as described above,
that SchurByPParent[i] is a list of length $j$, and that SchurByPChildren[i] 
is a list of length $j-1$. It appends these lists by adjoining the lists
SchurByP[i][j+1] and SchurByPParent[i][j+1] and SchurByPChildren[i][j].

By assumption each group $H$ in SchurByP[i][j] has $p$-class $j$. By Section 6 
of the article the desired $K$ are therefore precisely the descendants of $H$ 
of $p$-class $j+1$ with step size $$\dim \ker(K\twoheadrightarrow H) = \operatorname{MultiplicatorRank}(H)-2$$ and 
which possess the structure of a $\sigma$-$p$-group generated by odd elements.

The first macro computes these for a given $H$:

In [83]:
ComputeDescendants := function(j,H)
local ss,KList,K;
ss := MultiplicatorRank(H)-2;
# If the step size is 0, there is no such descendant.
if ss=0 
then return [];
else
KList := [];
# The ANUPQ function PqDescendants requires a step size >0:
for K in PqDescendants(H: ClassBound:=j+1, StepSize:=ss) do
if IsOurSigmaGroup(K)
then KList := Concatenation(KList,[K]);
fi; od;
return KList;
fi;
end;;

The second macro does this for all H in SchurByP[i][j] 
and records the information on descendants.

In [84]:
ComputeNextLevel := function(i,j)
local NextChildren,k,NewKList,NewParent,l;
Print("Computing p-class ",j+1," quotients for all G of type [243,",i,"]\n");
SchurByP[i][j+1] := [];
SchurByPParent[i][j+1] := [];
SchurByPChildren[i][j] := [];
for k in [1..Length(SchurByP[i][j])] do
# Record the new groups at level j+1:
NewKList := ComputeDescendants(j,SchurByP[i][j][k]);
SchurByP[i][j+1] := Concatenation(SchurByP[i][j+1],NewKList);
# Record their respective parents:
NewParent := [1..Length(NewKList)];
for l in [1..Length(NewKList)] do 
NewParent[l] := k; 
od;
SchurByPParent[i][j+1] := Concatenation(SchurByPParent[i][j+1],NewParent);
# Record the new groups as children of their parent:
NextChildren := 
   [Length(SchurByP[i][j+1])-Length(NewKList)+1..Length(SchurByP[i][j+1])];
SchurByPChildren[i][j] := 
   Concatenation(SchurByPChildren[i][j],[NextChildren]);
od;
end;;

For later use we compute all $G/P_j(G)$ for $j\le 5$:

In [89]:
TimeUsed();
for i in OurIndices do ComputeNextLevel(i,3); od;
Print("This has used ",TimeUsed()," milliseconds.");
for i in OurIndices do ComputeNextLevel(i,4); od;
Print("This has used ",TimeUsed()," milliseconds.");

Computing p-class 4 quotients for all G of type [243,2]
Computing p-class 4 quotients for all G of type [243,3]
Computing p-class 4 quotients for all G of type [243,4]
Computing p-class 4 quotients for all G of type [243,5]
Computing p-class 4 quotients for all G of type [243,6]
Computing p-class 4 quotients for all G of type [243,7]
Computing p-class 4 quotients for all G of type [243,8]
Computing p-class 4 quotients for all G of type [243,9]
Computing p-class 4 quotients for all G of type [243,13]
Computing p-class 4 quotients for all G of type [243,14]
Computing p-class 4 quotients for all G of type [243,15]
Computing p-class 4 quotients for all G of type [243,17]
Computing p-class 4 quotients for all G of type [243,18]
This has used 4852 milliseconds.Computing p-class 
5 quotients for all G of type [243,2]
Computing p-class 5 quotients for all G of type [243,3]
Computing p-class 5 quotients for all G of type [243,4]
Computing p-class 5 quotients for all G of type [243,5]
Computing 

7

Summarize the number of groups G/Pj(G) at for each i:

In [90]:
for j in [3..5] do
Print("Number of groups at level ",j,":  ");
for i in OurIndices do Print(Length(SchurByP[i][j])," "); od;
Print("\n"); 
od;

Number of groups at level 3:  10 1 1 1 1 1 1 1 3 3 3 2 3 
Number of groups at level 4:  43 1 1 0 1 0 1 1 5 5 5 3 3 
Number of groups at level 5:  100 198 2 0 6 0 6 6 63 14 28 32 3 


===============================================================================

Part 8: Testing whether $E(G)$ is powerful for all $G$ with given $G/P_j(G)E_2(G)$.

The following two macros take as arguments an integer $j\ge 3$, a function $E$ 
which to any $p$-group $G$ associates a normal subgroup $E(G)$, and a $p$-group $H$ 
respectively a list of $p$-groups, assumed to be isomorphic to $G/P_j(G)E_2(G)$ 
for some weak Schur $\sigma$-group $G$ of type $(3,3)$. If that condition is
not met, the correctness of the result is not guaranteed. 

The macros output a pair $[o,d]$ where

--- $o=1$    if $E(G)$ is powerful for all such $G$,

--- $o=0$    if $E(G)$ is powerful for no such $G$,

--- $o=1/2$  if each case happens for some such $G$,

and $d$ is the maximal number of generators of $E(G)$ as a pro-$p$-group, if $o=1$.

The macros call each other recursively.

The first macro concerns a single group $H$ with $E_2(H)=1$ by assumption.
Thus $E(H)$ is powerful if and only if $E1(H)=1$. Therefore if $E1(H)\neq 1$, 
then $E(H)$ is not powerful; hence $E(G)$ is never powerful, 
so we already have the answer without further computation.

Otherwise we compute the $p$-cover $H^*$ of $H$ and the step size
   $$ss := \operatorname{MultiplicatorRank}(H) - \dim_{\mathbb{F}_p} E_2(H^*) - 2\,.$$
Then by Section 6 of the article the groups $G/P_{j+1}(G)E_2(G)$ with 
$G/P_j(G)E_2(G) \cong H$, up to isomorphism, are precisely the descendants $K$ 
of $H$ of $p$-class $\le j+1$ and $\dim \ker(K\twoheadrightarrow H) = ss$, which also satisfy $E_2(K)=1$ and possess the structure of a $\sigma$-$p$-group generated by odd elements.

If the step size is $0$, then the only such descendant is $H$ itself.
In that case $E_1(H)$ already gives the complete answer.

Otherwise the first macro computes all possibilities for $K$ and
calls the second macro with that list and with $j+1$ in place of $j$.

The second macro simply applies the first macro to each member of the list
and compares the results.

This is just to avoid an error message and will be redefined below:

In [91]:
IsEGPowerfulList := function(j,E,HList) end;;

In [92]:
IsEGPowerfulOnce := function(j,E,H)
local Hstar,ss,KList,K;
Print("Beginning new group of log3-size ",
      LSize(H)," at level ",j," ...\n");
# If E(H) is not powerful, then E(G) is never powerful:
if Size(ComputeE1(E,H))>1
then Print("... not powerful\n");
return [0,0];
else
# Compute the step size for the next level:
Hstar := PqPCover(H: Prime:=3);
ss := MultiplicatorRank(H) - LSize(ComputeE2(E,Hstar)) - 2;
# If there are no proper such descendants, then E(G) is always powerful:
if ss=0 
then Print("... powerful\n");
return [1,RankPGroup(E(H))];
else 
# Generate all K:
KList := [];
# Here the step size ss is already >0, so we only need proper descendants.
for K in PqDescendants(H: StepSize:=ss) do
if Size(ComputeE2(E,K))=1
then if IsOurSigmaGroup(K)
then KList := Concatenation(KList,[K]);
fi; fi; od;
# Now test all K:
return IsEGPowerfulList(j+1,E,KList);
fi; fi;
end;;

In [93]:
IsEGPowerfulList := function(j,E,HList)
local yeshappens,nohappens,RankMax,H,answer;
# Initialize
yeshappens := false;
nohappens := false;
RankMax := 0;
for H in HList do
answer := IsEGPowerfulOnce(j,E,H);
if answer[1] > 0 then yeshappens := true; fi;
if answer[1] < 1 then nohappens := true; fi;
if answer[2] > RankMax then RankMax := answer[2]; fi;
# If both cases have occurred so far, we do not need to compute further:
if yeshappens and nohappens then return [1/2,0]; fi;
od;
# Now return the final answer:
if yeshappens 
then if nohappens 
then Print("Logical error","\n"); 
else return [1,RankMax];
fi;
else if nohappens
then return [0,0];
else Print("Problem: Empty list.\n");
fi; fi;
end;;

The following macro begins with $j$ and $E$ as above and a list of $p$-groups $H$
of the form $G/P_j(G)$ for a weak Schur $\sigma$-group $G$ of type $(3,3)$.
It applies the preceding macro to all $H/E_2(H)$.

In [94]:
IsEGPowerful := function(j,E,HList)
local H,KList,K,K1,notyet,answer;
# First create a list of all groups H/E2(H) without repetitions:
KList := [];
for H in HList do
# Make sure that GAP sees this as a Pq-group:
K := Pq(H/ComputeE2(E,H): Prime:=3);
# Test whether H/E2(H) is already in the list:
notyet := true;
for K1 in KList do
if IsPqIsomorphicPGroup(K,K1) 
then notyet := false;
break;
fi; od;
# If it wasn't already there, add it to the list:
if notyet
then KList := Concatenation(KList,[K]);
fi; od;
# Print(Length(HList)," ",Length(KList),"\n");
# Now test all possibilities for H/E2(H):
return IsEGPowerfulList(j,E,KList);
end;;

===============================================================================

Part 9: Testing whether $D_2(G)$ is powerful for all $G$

For each of the 13 cases we apply the last macro from Part 6 above:

In [95]:
for i in OurIndices do 
answer := IsEGPowerful(3,ComputeD2,SchurByP[i][3]);
Print("\nFor G of type [243,",i,"] the subgroup D2(G) is ");
if answer[1]=1 
then Print("always powerful of rank <= ",answer[2],".\n\n");
else if answer[1]=0
then Print("never powerful.\n\n");
else Print("sometimes powerful, sometimes not.\n\n");
fi; fi;
od;

Beginning new group of log3-size 5 at level 3 ...
... powerful

For G of type [243,2] the subgroup D2(G) is always powerful of rank <= 3.

Beginning new group of log3-size 5 at level 3 ...
Beginning new group of log3-size 6 at level 4 ...
Beginning new group of log3-size 8 at level 5 ...
... not powerful
Beginning new group of log3-size 8 at level 5 ...
... not powerful
Beginning new group of log3-size 8 at level 5 ...
... not powerful
Beginning new group of log3-size 8 at level 5 ...
... not powerful

For G of type [243,3] the subgroup D2(G) is never powerful.

Beginning new group of log3-size 5 at level 3 ...
... powerful

For G of type [243,4] the subgroup D2(G) is always powerful of rank <= 3.

Beginning new group of log3-size 5 at level 3 ...
... powerful

For G of type [243,5] the subgroup D2(G) is always powerful of rank <= 3.

Beginning new group of log3-size 5 at level 3 ...
... powerful

For G of type [243,6] the subgroup D2(G) is always powerful of rank <= 3.

Beginning new 

Thus for 10 of the 13 types, namely all $i$ in $[2,4,5,6,7,8,14,15,17,18]$,
the subgroup $D_2(G)$ is always powerful  of rank at most 3. For the other
3 types with $i$ in $[3,9,13]$ the subgroup $D_2(G)$ is never powerful.

In [97]:
OurBadIndices := [3,9,13];
Print("The computation for D2 has used ",TimeUsed()," milliseconds.");

The computation for D2 has used 1214 milliseconds.

[ 3, 9, 13 ]

===============================================================================

Part 10: Testing whether $D_3(G)$ is powerful for the remaining $G$:

First we test this for all groups of a given bad type $[243,i]$:

In [99]:
for i in OurBadIndices do 
answer := IsEGPowerful(3,ComputeD3,SchurByP[i][3]);
Print("\nFor G of type [243,",i,"] the subgroup D3(G) is ");
if answer[1]=1 
then Print("always powerful of rank <= ",answer[2],".\n\n");
else if answer[1]=0
then Print("never powerful.\n\n");
else Print("sometimes powerful, sometimes not.\n\n");
fi; fi;
od;
Print("The computation for D3 has used ",TimeUsed()," milliseconds.");

Beginning new group of log3-size 5 at level 3 ...
Beginning new group of log3-size 7 at level 4 ...
Beginning new group of log3-size 9 at level 5 ...
Beginning new group of log3-size 11 at level 6 ...
... not powerful
Beginning new group of log3-size 9 at level 5 ...
Beginning new group of log3-size 10 at level 6 ...
... not powerful
Beginning new group of log3-size 9 at level 5 ...
... powerful

For G of type [243,3] the subgroup D3(G) is sometimes powerful, sometimes not.

Beginning new group of log3-size 5 at level 3 ...
Beginning new group of log3-size 6 at level 4 ...
Beginning new group of log3-size 8 at level 5 ...
Beginning new group of log3-size 9 at level 6 ...
... not powerful
Beginning new group of log3-size 8 at level 5 ...
Beginning new group of log3-size 9 at level 6 ...
... not powerful
Beginning new group of log3-size 8 at level 5 ...
Beginning new group of log3-size 9 at level 6 ...
... not powerful
Beginning new group of log3-size 8 at level 5 ...
Beginning new group

So for $H_13$ the group $D_3(G)$ is never powerful, while for $H_3$ and $H_9$ both 
cases occur. For those two we therefore look at level 5.

For $G$ of type $[243,3]$ there are 198 possibilities for $G/P_5(G)$. Since the outcome depends only on the isomorphism class of $G/E_2(G)$, and the subgroup $E_2(G/P_5(G))$ is non-trivial in all those cases, we first make a list of the isomorphism 
classes of $G/P_5(G)E_2(G)$ and group the $G/P_5(G)$ accordingly:

In [104]:
SchurByP5E2TypeH3 := [];
SchurByP5E2TypeH3Covers := [];
for H in SchurByP[3][5] do
# Make sure that GAP sees H/E2(H) as a Pq-group:
K := Pq(H/ComputeE2(ComputeD3,H): Prime:=3);
# Test whether H/E2(H) is already in the list:
notyet := true;
for j in [1..Length(SchurByP5E2TypeH3)] do
if IsPqIsomorphicPGroup(K,SchurByP5E2TypeH3[j]) 
then notyet := false;
SchurByP5E2TypeH3Covers[j] := Concatenation(SchurByP5E2TypeH3Covers[j],[H]);
break;
fi; od;
# If it wasn't already there, add it to the list:
if notyet
then SchurByP5E2TypeH3 := Concatenation(SchurByP5E2TypeH3,[K]);
SchurByP5E2TypeH3Covers := Concatenation(SchurByP5E2TypeH3Covers,[[H]]);
fi; od;
Length(SchurByP5E2TypeH3);
# This reduces the number of cases from 198 to 4.
# Computing whether two groups are isomorphic takes long:
Print("This computation has used ",TimeUsed()," milliseconds.");

This computation has used 13067 milliseconds.

[  ]

[  ]

4

We repeat the computation separately for those:

In [105]:
for k in [1..Length(SchurByP5E2TypeH3)] do 
K:=SchurByP5E2TypeH3[k];
answer := IsEGPowerful(5,ComputeD3,[K]); 
if answer[1]=1 
then Print("                                           ",
           "... D3(G) always powerful of rank <= ",answer[2],"\n");
else if answer[1]=0
then Print("                                           ",
           "... D3(G) never powerful.\n");
else Print("                                           ",
           "... D3(G) sometimes powerful, sometimes not.\n");
fi; fi;
od;

Beginning new group of log3-size 9 at level 5 ...
Beginning new group of log3-size 11 at level 6 ...
... not powerful
                                           ... D3(G) never powerful.
Beginning new group of log3-size 9 at level 5 ...
Beginning new group of log3-size 10 at level 6 ...
... not powerful
                                           ... D3(G) never powerful.
Beginning new group of log3-size 9 at level 5 ...
... powerful
                                           ... D3(G) always powerful of rank <\
= 6
Beginning new group of log3-size 9 at level 5 ...
... powerful
                                           ... D3(G) always powerful of rank <\
= 6


So the outcome depends only on the isomorphism class of $G/P_6(G)E_2(G)$.

For two of those $D_3(G)$ is never powerful, and for two it is always powerful.

In particular the answer depends only on the isomorphism class of $G/P_6(G)$.

For what it is worth, the two cases are distinguished by the orders
of their automorphism groups:

In [106]:
for k in [1..Length(SchurByP5E2TypeH3)] do 
K:=SchurByP5E2TypeH3[k];
Print(Factors(Size(AutomorphismGroup(K))),"\n");
od;

[ 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3 ]
[ 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3 ]
[ 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3 ]
[ 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3 ]


This therefore distinguishes the bad groups from the good ones.

For $G$ of type $[243,9]$ there are 6 possibilities for $G/P_5(G)$.
We repeat the computation separately for those:

In [107]:
for K in SchurByP[9][5]
do answer := IsEGPowerful(5,ComputeD3,[K]); 
if answer[1]=1 
then Print("                                           ",
           "... D3(G) always powerful of rank <= ",answer[2],"\n");
else if answer[1]=0
then Print("                                           ",
           "... D3(G) never powerful.\n");
else Print("                                           ",
           "... D3(G) sometimes powerful, sometimes not.\n");
fi; fi;
od;

Beginning new group of log3-size 8 at level 5 ...
Beginning new group of log3-size 9 at level 6 ...
... not powerful
                                           ... D3(G) never powerful.
Beginning new group of log3-size 8 at level 5 ...
Beginning new group of log3-size 9 at level 6 ...
... not powerful
                                           ... D3(G) never powerful.
Beginning new group of log3-size 8 at level 5 ...
Beginning new group of log3-size 9 at level 6 ...
... not powerful
                                           ... D3(G) never powerful.
Beginning new group of log3-size 8 at level 5 ...
Beginning new group of log3-size 9 at level 6 ...
... not powerful
                                           ... D3(G) never powerful.
Beginning new group of log3-size 8 at level 5 ...
Beginning new group of log3-size 9 at level 6 ...
... powerful
                                           ... D3(G) always powerful of rank <\
= 6
Beginning new group of log3-size 8 at level 5 ...
Beginning

So the outcome depends only on the isomorphism class of $G/P_6(G)$.
For the 5th of these 6 isomorphism classes the group $D_3(G)$ is always 
powerful, for the remaining 5 the group $D_3(G)$ is never powerful.

I have tried in vain to find some easy property that distinguishes 
the fifth group in the list SchurByP[9][5] from the others.

We collate a list of groups of the form $G/P_6(G)$ for which $D_2(G)$ and $D_3(G)$
are never powerful, and for which we want to study $D_4(G)$:

In [113]:
BadSchurByP5 := [1..imax];;
BadSchurByP5[3] := Concatenation(SchurByP5E2TypeH3Covers[1],
                                SchurByP5E2TypeH3Covers[2]);;
BadSchurByP5[9] := SchurByP[9][5]{[1,2,3,4,6]};;
BadSchurByP5[13] := SchurByP[13][5];;

# Their lengths are 108, 5, 63:
Print(Length(BadSchurByP5[3])," ",
      Length(BadSchurByP5[9])," ",
      Length(BadSchurByP5[13]));
Print("The last computations have used ",TimeUsed()," milliseconds.");

108 5 63The last computations have used 6746 milliseconds.

===============================================================================

Part 11: Testing whether $D_4(G)$ is powerful for the remaining $G$:

We begin with the type $[243,9]$:

In [115]:
for K in BadSchurByP5[9] do 
answer := IsEGPowerful(5,ComputeD4,[K]);
if answer[1]=1 
then Print("                                           ",
           "... D4(G) always powerful of rank <= ",answer[2],"\n");
else if answer[1]=0
then Print("                                           ",
           "... D4(G) never powerful.\n");
else Print("                                           ",
           "... D4(G) sometimes powerful, sometimes not.\n");
fi; fi;
od;
# So in the first 3 bad subcases the group D4(G) is always powerful.
# In the 4-th and 5-th bad subcase the group D4(G) is never powerful.

Print("The computation for D3 has used ",TimeUsed()," milliseconds.");

Beginning new group of log3-size 8 at level 5 ...
Beginning new group of log3-size 9 at level 6 ...
Beginning new group of log3-size 11 at level 7 ...
... powerful
Beginning new group of log3-size 11 at level 7 ...
... powerful
                                           ... D4(G) always powerful of rank <\
= 6
Beginning new group of log3-size 8 at level 5 ...
Beginning new group of log3-size 9 at level 6 ...
Beginning new group of log3-size 11 at level 7 ...
... powerful
Beginning new group of log3-size 11 at level 7 ...
... powerful
                                           ... D4(G) always powerful of rank <\
= 6
Beginning new group of log3-size 8 at level 5 ...
Beginning new group of log3-size 9 at level 6 ...
Beginning new group of log3-size 11 at level 7 ...
... powerful
Beginning new group of log3-size 11 at level 7 ...
... powerful
                                           ... D4(G) always powerful of rank <\
= 6
Beginning new group of log3-size 8 at level 5 ...
Beginning new 

Next we consider the type [243,13]:

Here there are 63 possibilities for $G/P_5(G)$. Since the outcome depends only 
on the isomorphism class of $G/E_2(G)$, and the subgroup $E_2(G/P_5(G))$ is 
non-trivial in some of those cases, we first make a list of the isomorphism 
classes of $G/P_5(G)E_2(G)$ and group the $G/P_5(G)$ accordingly:

In [120]:
BadSchurByP5E2TypeH13 := [];
BadSchurByP5E2TypeH13Covers := [];
for H in BadSchurByP5[13] do
# Make sure that GAP sees H/E2(H) as a Pq-group:
K := Pq(H/ComputeE2(ComputeD4,H): Prime:=3);
# Test whether H/E2(H) is already in the list:
notyet := true;
for j in [1..Length(BadSchurByP5E2TypeH13)] do
if IsPqIsomorphicPGroup(K,BadSchurByP5E2TypeH13[j]) 
then notyet := false;
BadSchurByP5E2TypeH13Covers[j] := 
   Concatenation(BadSchurByP5E2TypeH13Covers[j],[H]);
break;
fi; od;
# If it wasn't already there, add it to the list:
if notyet
then BadSchurByP5E2TypeH13 := Concatenation(BadSchurByP5E2TypeH13,[K]);
BadSchurByP5E2TypeH13Covers := Concatenation(BadSchurByP5E2TypeH13Covers,[[H]]);
fi; od;
Length(BadSchurByP5E2TypeH13);
# This reduces the number of cases from 63 to 27.
Print("This computation has used ",TimeUsed()," milliseconds.");

This computation has used 9952 milliseconds.

[  ]

[  ]

27

We try the computation separately for those:

In [ ]:
for K in BadSchurByP5E2TypeH13 do
answer := IsEGPowerful(5,ComputeD4,[K]); 
if answer[1]=1 
then Print("                                           ",
           "... D4(G) always powerful of rank <= ",answer[2],"\n");
else if answer[1]=0
then Print("                                           ",
           "... D4(G) never powerful.\n");
else Print("                                           ",
           "... D4(G) sometimes powerful, sometimes not.\n");
fi; fi;
od;

Beginning new group of log3-size 9 at level 5 ...
Beginning new group of log3-size 11 at level 6 ...
Beginning new group of log3-size 15 at level 7 ...


Error, reached the pre-set memory limit
(change it with the -o command line option) in
  LETTER_WORD_EREP_CACHEPOS := LETTER_WORD_EREP_CACHEPOS mod 3 + 1; at /opt/homebrew/Cellar/gap/4.13.0/libexec/lib/wordlett.gi:169 called from 
ERepLettWord( w ) at /opt/homebrew/Cellar/gap/4.13.0/libexec/lib/wordlett.gi:178 called from
NumberSyllables( rhs ) at /opt/homebrew/Cellar/gap/4.13.0/libexec/lib/rwspcsng.gi:498 called from
SetConjugate( col, i, j, rhs ); at /opt/homebrew/Cellar/gap/4.13.0/libexec/lib/rwspcgrp.gi:912 called from
SingleCollector_GroupRelators( efam, gens, rods, powersp, powersn, commpp, commpn, commnp, commnn, conjpp, conjpn, conjnp, conjnn, conflicts ) at /opt/homebrew/Cellar/gap/4.13.0/libexec/lib/rwspcgrp.gi:1149 called from
SingleCollectorByRelators( efam, gens, rels, [  ] ) at /opt/homebrew/Cellar/gap/4.13.0/libexec/lib/rwspcgrp.gi:1162 called from
...  at stream:12


: 

This computation runs into a memory overload after only 3 invocations
of the macro IsEGPowerfulOnce and without even getting to level 8.

I tried again for the last K in the list BadSchurByP5E2TypeH13
and got this development before interrupting the process:

   Beginning new group of log3-size 9 at level 5

   Beginning new group of log3-size 11 at level 6

   Beginning new group of log3-size 15 at level 7

   Beginning new group of log3-size 18 at level 8

   Beginning new group of log3-size 21 at level 9

   Beginning new group of log3-size 24 at level 10

Next we consider the type $[243,3]$:

Here $E_2(G/P_5(G))$ is always trivial, so we cannot reduce the 108 cases.
We try the computation separately for those:

In [ ]:
for K in BadSchurByP5[3] do
answer := IsEGPowerful(5,ComputeD4,[K]); 
if answer[1]=1 
then Print("                                           ",
           "... D4(G) always powerful of rank <= ",answer[2],"\n");
else if answer[1]=0
then Print("                                           ",
           "... D4(G) never powerful.\n");
else Print("                                           ",
           "... D4(G) sometimes powerful, sometimes not.\n");
fi; fi;
od;

Beginning new group of log3-size 11 at level 5 ...
Beginning new group of log3-size 13 at level 6 ...


Again this computation runs into memory overloads at the first group.
I got this development before interrupting the process:

   Beginning new group of log3-size 11 at level 5
   
   Beginning new group of log3-size 13 at level 6

===============================================================================